# Chapter 6. 記憶容量

この章では、記憶容量、すなわち力学系が有する過去の入力を保持する能力を評価する指標の計算方法を学びます。
特に記憶関数 (Memory Function; MF) と記憶容量 (Memory Capacity; MC) の２つを実装し、その使い方を学習します。

## 前書き

記憶関数・記憶容量はH. Jaeger<sup>[1]</sup>らによって提案された指標で、力学系が過去の入力をどれほど保持できるかを定量化します。
以下の式で表される $N$次元の入力あり力学系 $x[k]$ とある線形写像$g: \mathbb{R}^N \to \mathbb{R}$ による出力 $\hat{y}[k]$ を考えます。

$$
\renewcommand{\Tau}{\mathrm{T}}
\renewcommand{\Zeta}{\mathrm{Z}}
\begin{align*}
x[k+1] &= f \left(x[k],\zeta[k+1]\right) \\
\hat{y}[k] &= g (x[k])
.\end{align*}
$$

$g$は線形なので、ある結合パラメータ $W^\mathrm{out} \in \mathbb{R}^{N+1}$ を用いて出力 $\hat{y}[k]$ は次の式で表現できます。

$$
\begin{align*}
\hat{y}[k] &= \hat{w} [1 ; x[k]] \\
&= \hat{w} {[1 \quad x_1[k] \quad \cdots \quad x_{N}[k]]}^\top \\
&= \hat{w}_0 + \sum_{i=1}^{N} \hat{w}_i x_{i}[k]
.\end{align*}
$$

入力 $\zeta[k]$、内部状態 $x[k]$ ともに定常的、すなわちその平均や分散が時間によらず一定であると仮定します。
このとき記憶関数 $\mathrm{MF}[\tau]$ は特に $\tau~(\geq 0)$ ステップ前の入力 $\zeta^\tau[k]:=\zeta[k-\tau]$ を内部状態 $x[k]$ からどれほど再構成できるかを評価する指標で、次の式で定義されます。

$$
\begin{align*}
\mathrm{MF}[\tau] :=& \max_{\hat{w}} \rho^2[\zeta^\tau, \hat{y}] \\
=& \max_{\hat{w}} \frac{\mathrm{Cov}^2[\zeta^\tau, \hat{y}]}{\mathrm{Var}[\zeta^\tau]\mathrm{Var}[\hat{y}]}
,\end{align*}
$$

ここで $\rho$ は相関係数、 $\mathrm{Cov}$ は共分散、$\mathrm{Var}$ は分散を表します。
相関係数の絶対値は $1$ 以下であるので、以下の不等式が成り立ちます。

$$
\begin{align*}
0 \leq \mathrm{MF}[\tau] \leq 1
.\end{align*}
$$

最後に全過去入力に対する記憶関数の総和により、以下の式で記憶容量 $\mathrm{MC}$ は定義されます。

$$
\begin{align*}
\mathrm{MC} := \sum_{\tau=0}^{\infty} \mathrm{MF}[\tau]
.\end{align*}
$$

特に入力がi.i.d.、つまり各時刻で$\zeta[k]$の値が独立にサンプルされる場合、以下の不等式の成立が知られています (導出は発展課題)。

$$
\begin{align*}
\mathrm{MC} \leq r \leq N
,\end{align*}
$$

ここで $r$ は $x[k]$ の階数を表し、式が示すとおり、高々線形独立な成分の数にその記憶容量が制限されます。

## 演習問題と実演

ここからは演習問題とデモンストレーションに移ります。
前回と同じライブラリの他、前回の演習で実装した`ESN`・`Linear`が`import`により利用できます。
初めに次のセルを実行してください。

なお`ESN`・`Linear`の内部実装を再確認するには、`import inspect`以下の行をコメントアウトするか`...?? / ??...`を使用してください。

In [ ]:
import sys

import numpy as np
import scipy as sp

if "google.colab" in sys.modules:
    from google.colab import drive  # type: ignore

    if False:  # Set to True if you want to use Google Drive and save your work there.
        drive.mount("/content/gdrive")
        %cd /content/gdrive/My Drive/rc-bootcamp/
        # NOTE: Change it to your own path if you put the zip file elsewhere.
        # e.g., %cd /content/gdrive/My Drive/[PATH_TO_EXTRACT]/rc-bootcamp/
    else:
        pass
        %cd /content/
        !git clone --branch ja https://github.com/rc-bootcamp/rc-bootcamp.git
        %cd /content/rc-bootcamp/
else:
    sys.path.append(".")

from utils.reservoir import ESN, Linear
from utils.style_config import plt
from utils.tester import load_from_chapter_name
from utils.tqdm import tqdm, trange

test_func, show_solution = load_from_chapter_name("06_memory_capacity")


# Uncomment it to see the implementations of `Linear` and `ESN`.
# import inspect
# print(inspect.getsource(Linear))
# print(inspect.getsource(ESN))

# Or just use ??.../...?? (uncomment the following lines).
# Linear??
# ESN??

### 1. 特異値分解を用いた記憶関数の実装

まず記憶関数の実装を行いましょう。
$\mathrm{MF}[\tau]$ は$\zeta^\tau$ と $\hat{y}$ の相関係数の二乗の最大値として定義されます。
一方で相関係数の二乗は、決定係数 $\mathrm{R}^2[\zeta^\tau, \hat{y}]$ と一致する (導出は発展課題) ので以下の式が成り立ちます。

$$
\begin{align*}
\mathrm{MF}[\tau] &= \max_{\hat{w}} \mathrm{R}^2[\zeta^\tau, \hat{y}] \\
&= 1 - \min_{\hat{w}} \frac{\mathrm{E}[(\zeta^\tau - \hat{y})^2]}{\mathrm{Var}[\zeta^\tau]} \\
&= 1 - \frac{\min_{\hat{w}} \mathrm{MSE}(\zeta^\tau, \hat{y})}{\mathrm{Var}[\zeta^\tau]}
.\end{align*}
$$

今線型回帰を考えているので $\mathrm{MSE}$ を最小化する $\hat{w}$ は線型回帰により一意に導出され、$\mathrm{MF}[\tau]$ は $\hat{y}$ を用いない以下の形式で表現されます。

$$
\begin{align*}
\mathrm{MF}[\tau] &= \mathrm{R}^2[\zeta^\tau, x]
.\end{align*}
$$

この式は、明示的に過去時系列を再構成する $g$ ならびに $\hat{w}$ を計算せずとも、特異値分解 (Singular Value Decomposition; SVD) を用いて計算できます (導出は第2章 Q5.4. 参照)。
つまり $T$ ステップに渡る内部状態のダイナミクスを格納した説明変数行列 $X=[x[0];x[1];~\ldots;~x[T-1]]^\top \in \mathbb{R}^{T \times N}$ と、対応する目的変数行列 $\Zeta^\tau = [\zeta[-\tau],\zeta[-\tau+1],~\ldots,~\zeta[T-\tau-1]]^\top \in \mathbb{R}^{T \times 1}$ に関して、$X=U\Sigma V^\top$ と分解された後、以下の式で $\mathrm{MF}[\tau]$ は計算されます。

$$
\begin{align*}
\mathrm{MF}[\tau] &= \frac{\|U^\top \Zeta^\tau\|^2}{\|\Zeta^\tau\|^2}
,\end{align*}
$$

ただし $X, \Zeta^\tau$ ともに正規化 (平均が零となるように変換) されているものとします。

Q1.1.

説明変数$X\in\mathbb{R}^{T \times N}$ が与えられる。Xを正規化したのち、$X=U\Sigma V^\top$ とSVDを行い、$U\in\mathbb{R}^{T \times N}$ と$X$ の階数 $r$ を出力する関数 `calc_regression_and_rank`を実装せよ。

- `calc_regression_and_rank`
  - Argument(s):
    - `X`: `np.ndarray`
      - `shape`: `(..., t, n)`
      - `dtype`: `np.float64`
  - Return(s):
    - `U`: `np.ndarray`
      - `shape`: `(..., t, n)`
      - `dtype`: `np.float64`
    - `mask`: `np.ndarray`
      - `shape`: `(..., n)`
      - `dtype`: `np.boolean`
    - `rank`: `np.ndarray`
      - `shape`: `(...,)`
      - `dtype`: `np.int64`
  - $10^{2} \leq T \leq 10^{3}$
  - $1 \leq N \leq 10^{2}$

<details><summary>tips</summary>

- [`np.linalg.svd`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.svd.html)
- [`np.linalg.matrix_rank`](https://numpy.org/devdocs/reference/generated/numpy.linalg.matrix_rank.html)

</details>

In [ ]:
def calc_regression_and_rank(X):
    T, N = X.shape[-2:]
    X = ...  # TODO Calculate centered X.
    U, sigma, V = ...  # TODO Use `np.linalg.svd` to perform SVD.
    eps = np.finfo(X.dtype).eps
    sigma_sq_max = np.max(sigma * sigma, axis=-1, keepdims=True)
    eps = sigma_sq_max * (eps * max(T, N))
    mask = sigma > eps
    rank = ...  # TODO Calculate rank by counting number of singular values greater than `eps`.
    return U, mask, rank


test_func(calc_regression_and_rank, "01_01", multiple_output=True)
# show_solution("01_01")  # Uncomment it to see the solution.

Q1.2.

`calc_regression_and_rank`で得られた $U \in \mathbb{R}^{T\times N}$ を用いて、$\mathrm{MF}[\tau]$ を計算する関数 `calc_memory_function`を実装せよ。

- `calc_memory_function`
  - Argument(s):
    - `U`: `np.ndarray`
      - `shape`: `(..., t, n)`
      - `dtype`: `np.float64`
    - `mask`: `np.ndarray`
      - `shape`: `(..., n)`
      - `dtype`: `np.boolean`
    - `zeta`: `np.ndarray`
      - `shape`: `(..., t, 1)`
      - `dtype`: `np.float64`
  - Return(s):
    - `r2`: `np.ndarray`
      - `shape`: `(..., t, 1)`
      - `dtype`: `np.float64`
  - $10^{2} \leq T \leq 10^{3}$
  - $1 \leq N \leq 10^{2}$

In [ ]:
def calc_memory_function(U, mask, zeta):
    uzeta = ...  # TODO Calculate U^T * zeta.
    dot = ((uzeta * uzeta) * mask[..., None]).sum(
        axis=-2
    )  # Calculate dot product considering only components where mask is True.
    var = ...  # TODO Calculate variance of zeta.
    r2 = ...  # TODO Calculate R^2 (`dot` divided by `var`).
    return r2


test_func(calc_memory_function, "01_02")
# show_solution("01_02")  # Uncomment it to see the solution.

Q1.3. (Advanced)

- 相関係数 $\rho^2$ と決定係数 $\mathrm{R}^2$ の関係を導出し、それらの一致を確認せよ。
- 線形ESN、すなわち活性化関数がなく、かつ入力時系列がi.i.d.であるとき、$\tau$に対する$\mathrm{MF}[\tau]$ の単調減少性を示せ。
- 入力時系列がi.i.d.であるとき、$\mathrm{MC} \leq N$ を示せ。

### 2. ESNの記憶容量の計算

ここまで実装した`calc_regression_and_rank`と`calc_memory_function`を用いて、記憶容量を図示化してみましょう。
以下のセルはESNの記憶容量を計算し記憶関数をプロットします。
入力時系列は一様乱数 $\mathcal{U}([-1, 1])$ からサンプルし、複数のスペクトル半径 (Spectral Radius; SR) に対して図示化します。

In [ ]:
seed = 1234
dim = 50
t_washout = 1000
t_sample = 20000
t_total = t_washout + t_sample
display = True

rnd = np.random.default_rng(seed)
srs = np.array([0.1, 0.5, 0.9])
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)

net = ESN(dim, sr=srs[:, None], f=np.tanh, p=1, rnd=rnd)
# net = ESN(dim, sr=srs[:, None], f=lambda t: t, p=1, rnd=rnd)  # Linear reservoir.
# net.weight[:] = np.roll(np.eye(dim), 1, axis=0)  # Ring topology.

x0 = np.zeros((srs.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape))
for idx in trange(t_total, display=display):
    x = net(x, w_in(us[idx]))
    xs[idx] = x

taus = np.arange(0, 81)
r2s = None
U, mask, ranks = calc_regression_and_rank(xs[t_washout:].swapaxes(0, -2))
for idx, tau in enumerate(tqdm(taus, display=display)):
    zeta = us[t_washout - tau : t_total - tau]
    r2 = calc_memory_function(U, mask, zeta)[..., 0]
    if r2s is None:
        r2s = np.zeros((taus.shape[0], *r2.shape))
    r2s[idx] = r2
mcs = np.sum(r2s, axis=0)
labels = []
for sr, rank, mc in zip(srs, ranks, mcs, strict=True):
    labels.append(f"SR={sr:.2f}, rank={rank}, MC={mc:.2f}")

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(taus, r2s, "o-", label=labels)
ax.set_xlim(taus.min() - 0.5, taus.max() + 0.5)
ax.set_ylim(-0.05, 1.05)
ax.legend(
    loc="upper right",
    borderaxespad=0,
    ncol=1,
    fontsize=12,
    frameon=False,
)
ax.set_xlabel(r"$\tau$", fontsize=14)
ax.set_ylabel(r"$\mathrm{MF}[\tau]$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)

None

今度はESNのスペクトル半径を細かく設定し、記憶容量との関係を調べてみましょう。
以下のセルはESNの記憶容量を計算し、階数と記憶容量を同時に表示します (デフォルトではスペクトル半径を 0.0から1.5まで0.05刻みで設定されています)。
記憶容量が階数を必ず下回る点、スペクトル半径が1に近づくほど記憶容量が大きくなる点を確認してください。

In [ ]:
seed = 1234
dim = 50
t_washout = 1000
t_sample = 20000
t_total = t_washout + t_sample
display = True

rnd = np.random.default_rng(seed)
srs = np.linspace(0.0, 1.5, 31)
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)

net = ESN(dim, sr=srs[:, None], f=np.tanh, p=1, rnd=rnd)
# net = ESN(dim, sr=srs[:, None], f=lambda t: t, p=1, rnd=rnd)

x0 = np.zeros((srs.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape))
for idx in trange(t_total, display=display):
    x = net(x, w_in(us[idx]))
    xs[idx] = x

taus = np.arange(0, 81)
r2s = None
U, mask, ranks = calc_regression_and_rank(xs[t_washout:].swapaxes(0, -2))
for idx, tau in enumerate(tqdm(taus, display=display)):
    zeta = us[t_washout - tau : t_total - tau]
    r2 = calc_memory_function(U, mask, zeta)[..., 0]
    if r2s is None:
        r2s = np.zeros((taus.shape[0], *r2.shape))
    r2s[idx] = r2
mcs = np.sum(r2s, axis=0)

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(srs, mcs, "o-", label="MC")
ax.plot(srs, ranks, "o-", color="k", label="rank")
ax.set_xlim(srs.min() - 0.05, srs.max() + 0.05)
# ax.set_ylim(-0.1, dim + 0.1)
ax.legend(
    loc="upper left",
    borderaxespad=0,
    bbox_to_anchor=(1.025, 1.0),
    ncol=1,
    fontsize=12,
    frameon=False,
)
ax.set_xlabel(r"SR", fontsize=14)
ax.set_ylabel(r"$\mathrm{MC}$", fontsize=14)
ax.tick_params(axis="both", which="major", labelsize=12)

None

前の章で学んだ最大リアプノフ指数との関連を調べてみましょう。
次のセルはESNの最大リアプノフ指数も同時に計算し、MCとの関係を図示化します。
スペクトル半径を0.01刻みで2.0まで200点設定しているため、環境によっては実行に若干時間がかかる点に注意してください。

In [ ]:
seed = 12345
dim = 100
eps = 1e-4
t_washout = 1000
t_sample = 10000
t_total = t_washout + t_sample
ts = np.arange(-t_washout, t_sample)
display = True

rnd = np.random.default_rng(seed)
srs = np.linspace(0.01, 2.0, 200)
w_in = Linear(1, dim, bound=0.1, bias=0.0, rnd=rnd)

net = ESN(dim, sr=srs[:, None], f=np.tanh, p=1, rnd=rnd)
# net = ESN(dim, sr=srs[:, None], f=lambda t: t, p=1, rnd=rnd)

x0 = np.zeros((2, srs.shape[0], dim))
us = rnd.uniform(-1, 1, (t_total, 1))

x = x0
xs = np.zeros((t_total, *x0.shape[1:]))
lmbds = np.zeros((t_sample, srs.shape[0]))
for idx, t in enumerate(tqdm(ts, display=display)):
    if t == 0:
        pert = rnd.uniform(-1, 1, x[0].shape)
        pert = pert / np.linalg.norm(pert, axis=-1, keepdims=True)
        x[1] = x[0] + pert * eps
    x = net(x, w_in(us[idx]))
    xs[idx] = x[0]
    if t >= 0:
        x_org, x_per = x[0], x[1]
        x_diff = x_per - x_org
        d_post = np.linalg.norm(x_diff, axis=-1, keepdims=True)
        lmbd = np.log(np.abs(d_post / eps))
        x_per[:] = x_org + x_diff * (eps / d_post)
        lmbds[idx - t_washout] = lmbd[..., 0]

taus = np.arange(0, 81)
r2s = None
U, mask, ranks = calc_regression_and_rank(xs[t_washout:].swapaxes(0, -2))
for idx, tau in enumerate(tqdm(taus, display=display)):
    zeta = us[t_washout - tau : t_total - tau]
    r2 = calc_memory_function(U, mask, zeta)[..., 0]
    if r2s is None:
        r2s = np.zeros((taus.shape[0], *r2.shape))
    r2s[idx] = r2
mcs = np.sum(r2s, axis=0)


def get_maxima_and_minima(xs, **kwargs):
    id_maxima = sp.signal.find_peaks(xs, **kwargs)[0]
    id_minima = sp.signal.find_peaks(-xs, **kwargs)[0]
    return id_maxima, id_minima


fig, ax = plt.subplots(2, 1, figsize=(8, 8), gridspec_kw={"hspace": 0.05})
axl = ax[0]
axl.set_xlim(srs.min() - 0.01, srs.max() + 0.01)
axl.set_xticklabels([])
for idx, sr in enumerate(srs):
    id_maxima, id_minima = get_maxima_and_minima(xs[t_washout:, idx, 0])
    id_all = np.concatenate([id_maxima, id_minima])
    peaks = xs[t_washout:, idx, 0][id_all]
    axl.scatter(sr * np.ones(peaks.shape[0]), peaks, marker=".", s=0.01, color="k")
axl.tick_params(axis="both", which="major", labelsize=12)
axl.set_xlabel(r"$x_0[k]$", fontsize=14)
axl.set_yticks([-1.0, 0.0, 1.0])
axl.set_ylim(-1.1, 1.1)

axr = ax[0].twinx()
axr.plot(srs, lmbds.mean(axis=0), "o-", color="red", label="MLE")
axr.set_yticks([-0.2, 0.0, 0.2])
axr.set_ylim(-0.22, 0.22)
axr.set_ylabel(r"MLE: $\lambda$", fontsize=14)
axr.set_xticklabels([])
axr.tick_params(axis="both", which="major", labelsize=12)

ax[1].plot(srs, mcs, "o-", label="MC")
ax[1].set_xlim(srs.min() - 0.01, srs.max() + 0.01)
ax[1].set_xlabel(r"SR", fontsize=14)
ax[1].set_ylabel(r"$\mathrm{MC}$", fontsize=14)
ax[1].tick_params(axis="both", which="major", labelsize=12)

None

Q2.1. (Advanced)

- パラメータを様々に変更して挙動がどう変化するか観察せよ。特に活性化関数とMF/MCとの関係を調査せよ。
- 入力分布は上のデモンストレーションでは一様乱数を用いているが、他の分布（例えば正規分布やベルヌーイ分布）を用いても良い。入力分布を変更した場合、MF/MCはどう変化するか観察せよ。

## 引用

[1] Jaeger, H. (2001). *Short term memory in echo state networks*. GMD Forschungszentrum Informationstechnik. https://doi.org/10.24406/publica-fhg-291107